In [ ]:
import pandas as pd
import os
import sys

mod = sys.modules[__name__]
def excel_reader():
    # 해당 경로에 있는 .xlsx 포맷의 파일이름을 리스트로 가져오기
    path = "/home/u1026/tm_2020/"
    file_list = os.listdir(path)
    exfile_list = [i for i in file_list if os.path.splitext(i)[-1]==".xlsx"]

    # 파일이름을 split 해 해당 파일의 날짜 리스트로 가져오기
    fname_list = []
    for i in range(len(exfile_list)):
        fname_list += [exfile_list[i].split("-")[0].split("_")[1]]

    # exfile_list에 있는 엑셀파일 모두 읽어오기
    for k in range(len(exfile_list)):
        file_locate = path + exfile_list[k]
        setattr(mod, 'file_{}'.format(fname_list[k]), pd.read_excel(file_locate))

In [ ]:
excel_reader()

In [ ]:
import pandas as pd

columns_name = ['일자', '제목', '본문', '인물', '기관', '키워드', '특성추출(가중치순 상위 50개)', 'URL']
file_20200801 = file_20200801[columns_name]
file_20200802 = file_20200802[columns_name]

news_20200801 = file_20200801
news_20200802 = file_20200802

#'제목'열에서 na가 있으면 해당 행 삭제
news_20200801 = news_20200801.dropna(subset=['제목'])
news_20200802 = news_20200802.dropna(subset=['제목'])

#row번호 리셋하기
news_20200801 = news_20200801.reset_index()
news_20200802 = news_20200802.reset_index()


#제목 마지막에 공백을 넣어 본문과 합칠 때 자연스럽게 합치게 함
for i in range(0,len(news_20200801)):
    news_20200801['제목'][i] = news_20200801['제목'][i] + ' '
    
for i in range(0,len(news_20200802)):
    news_20200802['제목'][i] = news_20200802['제목'][i] + ' '


news_20200801['내용'] = news_20200801.iloc[:, 2:4].sum(1)
news_20200802['내용'] = news_20200802.iloc[:, 2:4].sum(1)

for i in range(0,len(news_20200801)):
    news_20200801['내용'][i] = news_20200801['내용'][i].replace(',','').replace('\n','').replace('.','').replace('"','').replace('!','').replace('(',' ').replace(')','').replace('?','').casefold()
#숫자 지우기
#news_20200801['내용'] = news_20200801['내용'].str.replace('[0-9]', '')

for i in range(0,len(news_20200802)):
    news_20200802['내용'][i] = news_20200802['내용'][i].replace(',','').replace('\n','').replace('.','').replace('"','').replace('!','').replace('(',' ').replace(')','').replace('?','').casefold()
#news_20200802['내용'] = news_20200802['내용'].str.replace('[0-9]', '')

In [ ]:
#word_tokenize : 단어 기준으로 토큰화(Tokenize)
from nltk.tokenize import word_tokenize

word_20200801 = []
for i in range(len(news_20200801)):
    word_20200801 += (word_20200801, word_tokenize(news_20200801['내용'][i]))

In [ ]:
from konlpy.tag import Komoran

#메모리 늘려주기
komoran = Komoran(max_heap_size=1024 * 6)

km_word_20200801 = []
for i in range(len(news_20200801)):
    #print(okt.morphs(news_190801['내용'][i]))
    km_word_20200801 +=  komoran.nouns(news_20200801['내용'][i])

In [ ]:
stop_words = []
with open('./data_files/stopwords_list1.txt', 'r') as file1, \
open('./data_files/stopwords_list2.txt', 'r') as file2:
    for text1 in file1:
        stop_words.append(text1.strip('\n'))
    print(len(stop_words))
    for text2_1 in file2:
        text2 = text2_1.split('\t')
        stop_words.append(text2[0].strip('\n'))
    print(len(stop_words))

#불용어리스트 생성
stopwords_set = set(stop_words)
stop_words = list(stopwords_set)


#불용어 제거하기
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

#komoran 사용할 때
word_result_20200801 = []
for w in km_word_20200801:
    if w not in stop_words:
        word_result_20200801.append(w)

print("----불용어 제거 전----")
print(len(km_word_20200801))
print("----불용어 제거 후----")
print(len(word_result_20200801))

In [ ]:
#vocabulary 만들기
print(len(word_result_20200801))
set_20200801 = set(word_result_20200801)
voca_20200801 = list(set_20200801)

#vocabulary 한글자이면 지우기
for i in voca_20200801:
    if len(i) == 1:
        voca_20200801.remove(i)

print(len(voca_20200801))

In [ ]:
#DTM
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
dtm_ar_20200801 = cv.fit_transform(voca_20200801).toarray()
feature_names = cv.get_feature_names()
#dtm을 데이터프레임 형태로
dtm_20200801 = pd.DataFrame(dtm_ar_20200801, columns=feature_names)